In [1]:
import numpy as np
from sashimi_c import *
import os

In [5]:
nk=100
nM=100
nz=10

k_grid = np.logspace(-5,1,nk)
kmin = k_grid.min()
kmax = k_grid.max()
print(f'kmin = {kmin:.2e} 1/Mpc, kmax = {kmax:.2e} 1/Mpc')

M_grid = np.logspace(9.0, 17.0, nM) #Masses at z=0

z_grid = np.linspace(0.0,2.0,nz)
zmin = z_grid.min()
zmax = z_grid.max()
print(f'zmin = {zmin:.2f}, zmax = {zmax:.2f}')

kmin = 1.00e-05 1/Mpc, kmax = 1.00e+01 1/Mpc
zmin = 0.00, zmax = 2.00


For a (truncated) NFW profile with characteristic density \( \rho_s \), scale radius
\( r_s \), and truncation radius \( r_t \) (here \( r_t = c_t\, r_s \)), the real-space
density is

$$
\rho(r) =
\begin{cases}
\dfrac{\rho_s}{(r/r_s)\,(1+r/r_s)^2}, & r \le r_t \\
0, & r > r_t \; .
\end{cases}
$$

The 3D Fourier transform of \( \rho(r) \) is

$$
\tilde{\rho}(k)
= 4\pi \int_0^{r_t} r^2 \rho(r)\,
\frac{\sin(kr)}{kr}\, \mathrm{d}r \; .
$$

If you want the mass-normalized halo-model profile \( u(k) \), divide by the mass
inside the truncation radius:

$$
u(k) = \frac{\tilde{\rho}(k)}{M(r_t)} ,
$$

where

$$
M(r_t)
= 4\pi \rho_s r_s^3
\left[\ln(1+c_t) - \frac{c_t}{1+c_t}\right],
\qquad
c_t \equiv \frac{r_t}{r_s}.
$$

-> Same function as for untruncated NFW, only replace $r_{200}/r_{vir}$ by the truncation radius $r_t$.

In [ ]:
def u_nfw(k, c, rs):
    k  = np.atleast_1d(k).astype(float)
    rs = np.atleast_1d(rs).astype(float)
    c  = np.atleast_1d(c).astype(float)

    if rs.shape != c.shape:
        raise ValueError("rs and c must have the same shape")

    # Broadcast k against halo parameters
    x = k[:, None] * rs[None, :]

    denom = np.log(1.0 + c) - c / (1.0 + c)
    denom = denom[None, :]  # broadcast over k

    # sine and cosine integrals
    Si_x,  Ci_x  = special.sici(x)
    Si_xc, Ci_xc = special.sici((1.0 + c)[None, :] * x)

    u = (
        np.cos(x) * (Ci_xc - Ci_x)
        + np.sin(x) * (Si_xc - Si_x)
        - np.sin(c[None, :] * x) / ((1.0 + c)[None, :] * x)
    ) / denom

    # Return scalar if everything was scalar
    if u.shape == (1, 1):
        return u[0, 0]
    return u

In [ ]:
dirname = 'subhalo_parameters003'
os.makedirs(dirname, exist_ok=True)

# Use float32 unless you truly need float64
m_mz  = np.empty((nM, nz), dtype=np.float32)
I_kmz = np.empty((nk, nM, nz), dtype=np.float32)
J_kmz = np.empty((nk, nM, nz), dtype=np.float32)

for M_id,M in enumerate(M_grid):
    for z_id in range(nz):
        print(f"{M_id+1}/{nM}  {z_id+1}/{nz}")

        obs = subhalo_observables(
            M0_per_Msun=M,
            redshift=z_grid[z_id],
            M0_at_redshift=False
        )

        w1 = obs.m0 * obs.weight
        w2 = obs.m0**2 * obs.weight

        m_mz[M_id, z_id] = np.sum(w1)

        for k_id, k in enumerate(k_grid):
            u_k = u_nfw(k, obs.ct0, obs.rs0)  

            I_kmz[k_id, M_id, z_id] = np.sum(w1 * u_k) / M
            J_kmz[k_id, M_id, z_id] = np.sum(w2 * u_k**2) / M**2

np.savez(
    os.path.join(dirname, 'subhalo_params.npz'),
    k_grid=k_grid,
    M_grid=M_grid,
    z_grid=z_grid,
    m_mz=m_mz,
    I_kmz=I_kmz,
    J_kmz=J_kmz
)

1/100  1/10
1/100  2/10
1/100  3/10
1/100  4/10
1/100  5/10
1/100  6/10
1/100  7/10
1/100  8/10
1/100  9/10
1/100  10/10
2/100  1/10
2/100  2/10
2/100  3/10
2/100  4/10
2/100  5/10
2/100  6/10
2/100  7/10
2/100  8/10
2/100  9/10
2/100  10/10
3/100  1/10
3/100  2/10
3/100  3/10
3/100  4/10
3/100  5/10
3/100  6/10
3/100  7/10
3/100  8/10
3/100  9/10
3/100  10/10
4/100  1/10
4/100  2/10
4/100  3/10
4/100  4/10
4/100  5/10
4/100  6/10
4/100  7/10
4/100  8/10
4/100  9/10
4/100  10/10
5/100  1/10
5/100  2/10
5/100  3/10
5/100  4/10
5/100  5/10
5/100  6/10
5/100  7/10
5/100  8/10
5/100  9/10
5/100  10/10
6/100  1/10
6/100  2/10
6/100  3/10
6/100  4/10
6/100  5/10
6/100  6/10
6/100  7/10
6/100  8/10
6/100  9/10
6/100  10/10
7/100  1/10
7/100  2/10
7/100  3/10
7/100  4/10
7/100  5/10
7/100  6/10
7/100  7/10
7/100  8/10
7/100  9/10
7/100  10/10
8/100  1/10
8/100  2/10
8/100  3/10
8/100  4/10
8/100  5/10
8/100  6/10
8/100  7/10
8/100  8/10
8/100  9/10
8/100  10/10
9/100  1/10
9/100  2/10
9/100  3